In [ ]:
#!pip install pandas 
#!pip install matplotlib
#!pip install scipy
#!pip install pandas seaborn matplotlib
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

import pandas as pd 
import  numpy as np
dist_file_1="/home/sophie/uncertain-identity-aware-tracking/deeplabcut/pig_tracking_dataset/evaluation-results/iteration-1/CDPQ_testFeb22-trainset90shuffle1/dist_100002.csv"
dist_file_2="/home/sophie/uncertain-identity-aware-tracking/deeplabcut/pig_tracking_dataset/evaluation-results/iteration-2/CDPQ_testFeb22-trainset90shuffle1/dist_10000.csv"
dist_file_3="/home/sophie/uncertain-identity-aware-tracking/deeplabcut/pig_tracking_dataset/evaluation-results/iteration-0/CDPQ_testFeb22-trainset90shuffle1/dist_10000.csv"


df_read_1 = pd.read_csv(dist_file_1, header=[0, 1, 2, 3], index_col=0)
df_read_2 = pd.read_csv(dist_file_2, header=[0, 1, 2, 3], index_col=0)
df_read_3 = pd.read_csv(dist_file_3, header=[0, 1, 2, 3], index_col=0)
df_read_1.index

Reading the rmse  and plotting the distribution

In [ ]:


def generate_arrays(df_read):
    # Step 2: Select all columns with 'rmse' at the last level
    rmse_columns = df_read.loc[:, pd.IndexSlice[:, :, :, 'rmse']]
    all_rmse=[]
    body_part=[]
    individuals=[]
    frames=[]
    for col in rmse_columns.columns:
        all_rmse= all_rmse+list(rmse_columns[col])
        body_part=body_part+[col[-2] for a in list(rmse_columns[col])]
        individuals=individuals+[col[-3] for a in list(rmse_columns[col])]
        frames=frames+list(df_read.index)
    # Print the selected columns
    print("\nColumns with 'rmse' at the last level:")
    def to_float(value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return np.nan

    all_rmse = [to_float(a) for a in all_rmse ]
    all_rmse = np.array(all_rmse, dtype=float)
    body_part=np.array(body_part)
    individuals=np.array(individuals)
    frames=np.array(frames)
    body_part=body_part[~np.isnan(all_rmse)]
    individuals=individuals[~np.isnan(all_rmse)]
    frames=frames[~np.isnan(all_rmse)]

    all_rmse = all_rmse[~np.isnan(all_rmse)]
    print(len(all_rmse), np.mean(all_rmse), len(body_part))
    data= {'value':all_rmse, "body_part":body_part, "individuals":individuals, "frames":frames}
    df=pd.DataFrame(data)
    return df


df= generate_arrays (df_read_1)
df= generate_arrays (df_read_2)
df= generate_arrays (df_read_3)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Boxplot
sns.boxplot(x='body_part', y='value', data=df, ax=axes[0])
axes[0].set_title('Boxplot of body part rmse')

"""# Density Plot
sns.kdeplot(data=df, x='value', hue='body_part', fill=True, ax=axes[1] )
axes[1].set_title('Density of body part rmse')
"""
# Histogram
sns.histplot(data=df, x='value', hue='body_part', multiple='stack', ax=axes[1], bins=50, stat="percent")#, kde=True)
axes[1].set_title('Histogram of body part rmse in percentage')

plt.tight_layout()
plt.show()
#rmse full:6.54, rmse center-tail:2.93, rmse 2 ears head:7.06

 number and percentage  of individuals having at least one bodypart far 

In [47]:
df
df['value_above_100'] = df['value'] > 100

# Group by frames and calculate the number of unique individuals with value > 100
frames_grouped = df.groupby('frames').apply(lambda x: x[x['value_above_100']]['individuals'].nunique())

frames_grouped_all_far_point_per_frame=  df.groupby('frames').apply(lambda x: x[x['value_above_100']]['individuals'])
# Calculate the total number of unique individuals per frame
total_individuals_per_frame = df.groupby('frames')['individuals'].nunique()

# Calculate the percentage
percentage_individuals_above_100 = (frames_grouped / total_individuals_per_frame) *100

# Combine results into a DataFrame
result_df = pd.DataFrame({
    'count_unique_individuals_with_body_part_above_100': frames_grouped,
    'total_individuals': total_individuals_per_frame,
    'percentage_unique_individuals_with_body_part_above_100_per_frame': percentage_individuals_above_100
}).reset_index()

result_df.mean()


frames                         393.500000
count_individuals_above_100      0.293147
total_individuals               15.000000
percentage_above_100             1.954315
dtype: float64

In [36]:
pd.set_option('display.max_rows', 20)
frames_grouped_all_far_point_per_frame=  df.groupby('frames').apply(lambda x: x[x['value_above_100']])

frames_grouped_all_far_point_per_frame

value body_part   individuals  frames  value_above_100
frames                                                                   
18     41690  146.148244      lear   individual9      18             True
23     7679   100.034776      tail  individual11      23             True
25     38651  300.792151      lear   individual8      25             True
28     918    241.582650      lear   individual1      28             True
       9336   155.184912      lear  individual12      28             True
...                  ...       ...           ...     ...              ...
776    25713  207.898874      tail   individual3     776             True
777    5664   175.717549      tail  individual10     777             True
778    37690  174.490971      tail   individual7     778             True
785    1929   300.796798      rear   individual1     785             True
786    4907   304.042133      rear  individual10     786             True

[233 rows x 5 columns]

In [39]:
pd.set_option('display.max_rows', None)
frames_grouped = df.groupby('frames').apply(lambda x: x[x['value_above_100']]['individuals'].nunique())
frames_grouped

frames
0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     1
24     0
25     1
26     0
27     0
28     2
29     3
30     0
31     0
32     0
33     0
34     1
35     0
36     0
37     1
38     0
39     0
40     0
41     0
42     0
43     0
44     0
45     1
46     0
47     3
48     0
49     0
50     0
51     1
52     1
53     0
54     0
55     0
56     0
57     0
58     0
59     0
60     0
61     0
62     0
63     1
64     0
65     1
66     0
67     1
68     0
69     0
70     0
71     1
72     0
73     0
74     0
75     0
76     1
77     0
78     1
79     1
80     0
81     1
82     0
83     0
84     0
85     1
86     2
87     1
88     1
89     0
90     1
91     0
92     1
93     0
94     0
95     0
96     0
97     0
98     0
99     0
100    0
101    1
102    0
103    0
104    0
105    0
106    0
107    1
108    0
109    0
110